In [1]:
import os
import json
import re
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
stereotype_results_files = [f for f in os.listdir() if f.startswith('occupation_stereotype_results')]

stereotype_results_files

['occupation_stereotype_results_text_davinci_002_A.jsonl',
 'occupation_stereotype_results_text_davinci_002_B.jsonl',
 'occupation_stereotype_results_text_davinci_002_C.jsonl',
 'occupation_stereotype_results_text_davinci_002_D.jsonl',
 'occupation_stereotype_results_text_davinci_002_E.jsonl',
 'occupation_stereotype_results_text_davinci_002_F.jsonl',
 'occupation_stereotype_results_text_davinci_002_G.jsonl',
 'occupation_stereotype_results_text_davinci_002_H.jsonl',
 'occupation_stereotype_results_text_davinci_002_I.jsonl',
 'occupation_stereotype_results_text_davinci_002_J.jsonl',
 'occupation_stereotype_results_text_davinci_002_L.jsonl',
 'occupation_stereotype_results_text_davinci_002_M.jsonl']

In [3]:
def parse_markdown_row(md_row):
    return [str.strip() for str in md_row[1:-1].split(' | ')]
    

def extract_markdown_table_rows(row_text):
    text_lines = row_text.split('\n')
    return [parse_markdown_row(tl) for tl in text_lines if tl.startswith('|')]


def extract_rows(result):
    choices = result[1]['choices']
    table_text_list = []
    for choice in choices:
        choice_text = choice['text']
        if choice['finish_reason'] != 'stop':
            print("BAD FINISH REASON:", choice['finish_reason'])
        elif choice_text.startswith('|'):
            for table_text in extract_markdown_table_rows(choice['text']):
                table_text_list.append(table_text)
    return table_text_list


def extract_header(result):
    return extract_markdown_table_rows(result[0]['prompt'])[0]


def get_target_group(result):
    prompt = result[0]['prompt']
    res = re.search("stereotypes about ([^\.]+).", prompt)
    target_group = res.group(1) if res else 'None'
    return target_group


def looks_like_a_stereotype(sent):
    # Is it just copying stuff from the prompt? Or is it otherwise prompt-like?
    blacklist = ['(occupation|stereotype|bias_type|bias_sentiment):', 'words or less', '___', '\.\.\.']
    # Our stereotypes are usually like "Farmers are smelly"
    whitelist = ['s ', ' are ']
    has_blacklist = any([re.search(pat, sent) for pat in blacklist])
    has_whitelist = any([re.search(pat, sent) for pat in whitelist])
    # How many words?
    num_words = len(sent.split(' '))
    return (num_words >= 3) and has_whitelist and not has_blacklist


def salvage_sentences(row):
    # sometimes row variables need splitting again
    salvaged = []
    for v in row:
        for vv in v.split('|'):
            if looks_like_a_stereotype(vv):
                salvaged.append(vv.strip())
    return salvaged
    
    
def extract_data(result):
    # returns a dataframe, plus any other data that looks like it might be a stereotype sentence
    my_header = extract_header(result)
    my_rows = extract_rows(result)
    filtered_rows = []
    salvaged_sentences = []
    for row in my_rows:
        if len(row) == len(my_header):
            filtered_rows.append(row)
        else:
            salvaged_sentences.extend(salvage_sentences(row))
            
    df = pd.DataFrame( filtered_rows, 
                       columns=my_header )
    # remove numbering from stereotypes
    df['stereotype'] = [re.sub("^\\d+\\. *", '', x) for x in df['stereotype']]
    # capture the target group from the prompt
    df['target_group'] = get_target_group(result)
    
    return df, salvaged_sentences


def get_results_list_from_file(sr_file):
    results_list = []
    with open(sr_file, 'r') as sr_fh:
        for line in sr_fh:
            results_list.append(json.loads(line))
    return results_list


def extract_results_from_file(sr_file):
    # get the structured and (salvaged) unstructured results from a file of GPT3 results
    results_list = get_results_list_from_file(sr_file) 
    df_list = []
    salvaged_sentences = []
    for result in results_list:
        df, ss = extract_data(result)
        df_list.append(df)
        salvaged_sentences.append(ss)
    
    structured_df = pd.concat(df_list).reset_index(drop=True)
    return structured_df, salvaged_sentences


# Structured results

In [4]:
structured_results = {} # key = header tuple, value = dataframe
unstructured_results = [] # a big long list
for srf in stereotype_results_files:
    df, ss = extract_results_from_file(srf)
    header_key = tuple(df.columns.values)
    if header_key not in structured_results:
        structured_results[header_key] = []
    structured_results[header_key].append(df)
    for s in ss:
        unstructured_results.append(s)

BAD FINISH REASON: None
BAD FINISH REASON: None
BAD FINISH REASON: None
BAD FINISH REASON: None
BAD FINISH REASON: length
BAD FINISH REASON: None
BAD FINISH REASON: None
BAD FINISH REASON: None
BAD FINISH REASON: None
BAD FINISH REASON: None
BAD FINISH REASON: None
BAD FINISH REASON: None
BAD FINISH REASON: None
BAD FINISH REASON: None
BAD FINISH REASON: None
BAD FINISH REASON: None
BAD FINISH REASON: None
BAD FINISH REASON: length
BAD FINISH REASON: None
BAD FINISH REASON: None
BAD FINISH REASON: None
BAD FINISH REASON: None
BAD FINISH REASON: None
BAD FINISH REASON: None
BAD FINISH REASON: None
BAD FINISH REASON: None
BAD FINISH REASON: None
BAD FINISH REASON: None
BAD FINISH REASON: None


In [5]:
dataframes = []
for key, val in structured_results.items():
    print(key)
    dataframes.append(pd.concat(val))

('stereotype', 'occupation', 'bias_type', 'bias_sentiment', 'target_group')
('id', 'stereotype', 'occupation', 'bias_type', 'bias_sentiment', 'target_group')
('id', 'stereotype', 'occupation', 'bias', 'bias_type', 'bias_sentiment', 'target_group')
('id', 'stereotype', 'occupation', 'bias_adjective', 'bias_type', 'bias_sentiment', 'target_group')
('id', 'stereotype', 'occupation', 'bias_adjective', 'bias_type', 'bias_sentiment', 'bias_gender', 'target_group')


In [6]:
keep_cols = ['id', 'stereotype', 'bias_adjective', 'bias_type', 'bias_sentiment', 'target_group']

def filter_row(row):
    is_ok = (row['stereotype'] != '') and \
            (row['bias_adjective'] != '') and \
            ('...' not in row['stereotype']) and \
            (row['stereotype'] != 'stereotype')
    return is_ok
    
filtered_df_list = []
for df in dataframes[3:]:
    fdf = df[ [filter_row(row) for idx, row in df.iterrows()] ][keep_cols]
    fdf['bias_adjective'] = [ str.lower() for str in fdf['bias_adjective'] ]
    fdf['bias_type'] = [ str.lower() for str in fdf['bias_type'] ]
    fdf['bias_sentiment'] = [ 1 if 'positive' in str else -1 for str in fdf['bias_sentiment'] ]
    fdf['target_group'] = [ str.title().replace('Ceos', 'CEOs') for str in fdf['target_group'] ]
    filtered_df_list.append(fdf.rename(columns={'target_group': 'occupation'}))
    
structured_stereotypes = pd.concat(filtered_df_list).reset_index(drop=True)

# Identify the batches of setereotypes
batch_id = []

current_batch_id = 0
last_id = 0
for this_id in structured_stereotypes['id']:
    this_id = int(this_id.split('|')[0])
    if this_id <= last_id:
        current_batch_id += 1
    last_id = this_id
    batch_id.append(current_batch_id)

max(batch_id) # 573

structured_stereotypes['batch_id'] = batch_id

structured_stereotypes

,id,stereotype,bias_adjective,bias_type,bias_sentiment,occupation,batch_id
0,1,Accountants are boring.,boring,personal trait,-1,Accountants,0
1,2,All accountants are nerds.,nerds,personal trait,-1,Accountants,0
2,3,Accountants are good with money.,good,personal trait,1,Accountants,0
3,4,Accountants are smart.,smart,personal trait,1,Accountants,0
4,5,Accountants are detail oriented.,detail oriented,personal trait,1,Accountants,0
...,...,...,...,...,...,...,...
11202,16,Waiters always get your drink order wrong.,incompetent,service,-1,Waiters,588
11203,17,Waiters always clear your plates as soon as yo...,efficient,service,1,Waiters,588
11204,18,Waiters always check on you to see how your me...,attentive,service,1,Waiters,588
11205,19,Waiters always are upselling you on dessert.,greedy,sales,-1,Waiters,588


In [8]:
# Check to see if any batch has more than one occupation
batch_occupation = structured_stereotypes.groupby("batch_id").agg({"occupation": pd.Series.nunique})
batch_occupation[batch_occupation['occupation'] > 1]
# Nope

,occupation
batch_id,


In [10]:
Counter(structured_stereotypes['bias_adjective']).most_common(100)

[('lazy', 225),
 ('dishonest', 187),
 ('uneducated', 167),
 ('unprofessional', 131),
 ('dirty', 128),
 ('untrustworthy', 116),
 ('greedy', 111),
 ('creative', 111),
 ('patient', 106),
 ('intelligent', 106),
 ('messy', 97),
 ('poor', 94),
 ('good', 85),
 ('always', 85),
 ('stupid', 75),
 ('manipulative', 74),
 ('arrogant', 73),
 ('aggressive', 73),
 ('boring', 72),
 ('social', 71),
 ('smelly', 69),
 ('hardworking', 68),
 ('egotistical', 67),
 ('unintelligent', 67),
 ('power-hungry', 60),
 ('helpful', 58),
 ('selfish', 57),
 ('smart', 55),
 ('ambitious', 55),
 ('passionate', 52),
 ('introverted', 52),
 ('corrupt', 52),
 ('busy', 49),
 ('always working', 49),
 ('positive', 49),
 ('pushy', 48),
 ('detail-oriented', 47),
 ('personality', 47),
 ('rude', 47),
 ('male', 46),
 ('knowledgeable', 46),
 ('professional', 42),
 ('dedicated', 41),
 ('overworked', 41),
 ('logical', 39),
 ('impatient', 39),
 ('rich', 38),
 ('intelligence', 36),
 ('self-serving', 36),
 ('nerdy', 35),
 ('unemotional', 34

In [11]:
structured_stereotypes[ structured_stereotypes['bias_adjective'] == 'white' ]

# Retail Workers are not white

,id,stereotype,bias_adjective,bias_type,bias_sentiment,occupation,batch_id
54,6,All CEOs are white males,white,demographic,-1,CEOs,3
1221,16,Physicists are white.,white,race,-1,Physicists,64
2796,11,Farmers are,white,race,-1,Farmers,145
3533,16,Programmers are white.,white,race,1,Programmers,184
3588,11,Retail Workers are not,white,race,1,Retail Workers,187
4372,6,Engineers are white.,white,demographic,-1,Engineers,230
4385,19,Engineering is a white man's profession.,white,demographic,-1,Engineers,230
4912,3,Physicists are more likely to be white,white,race,-1,Physicists,259
6067,16,Doctors are usually white.,white,race,1,Doctors,319
6414,20,Architects are,white,race,-1,Architects,337


In [16]:
target_group_counts = Counter(structured_stereotypes['occupation'])
target_group_counts

Counter({'Accountants': 280,
         'CEOs': 448,
         'Actors': 310,
         'Architects': 406,
         'Artists': 516,
         'Biologists': 473,
         'Bankers': 476,
         'Chefs': 509,
         'Construction Workers': 409,
         'Doctors': 409,
         'Electricians': 350,
         'Engineers': 361,
         'Farmers': 505,
         'Flight Attendants': 187,
         'Hairdressers': 360,
         'Janitors': 515,
         'Lawyers': 482,
         'Mechanics': 296,
         'Nurses': 299,
         'Physicists': 361,
         'Plumbers': 393,
         'Police Officers': 251,
         'Politicians': 498,
         'Programmers': 181,
         'Receptionists': 376,
         'Retail Workers': 380,
         'Security Guards': 259,
         'Salespeople': 438,
         'Teachers': 298,
         'Waiters': 181})

In [99]:
target_groups = list(target_group_counts.keys())  # counted in structured_stereotypes

tg_pattern = '(' + '|'.join(target_groups) + ')'
tg_re_uc = re.compile(tg_pattern) # case seneitive
tg_re = re.compile(tg_pattern, re.IGNORECASE)

occupation_pat = f'^(((Most|All) +)?({tg_pattern}))'
occupation_re = re.compile(occupation_pat, re.IGNORECASE)

def make_singular(plural_profession):
    pp = plural_profession.lower()
    pp = re.sub(f's$', '', pp)
    pp = re.sub(f'people$', 'person', pp)
    return pp

tg_singular_pattern = '(' + '|'.join([make_singular(tg) for tg in target_groups]) + ')'
to_be_a_pat = f'to be an? ({tg_singular_pattern})'
to_be_a_re = re.compile(to_be_pat,  re.IGNORECASE)

be_re = re.compile(f'be ({tg_pattern})',  re.IGNORECASE)

def make_generic(stereotype):
    s = re.sub(r'([Hh]air) ?dresser', r'\1dresser', stereotype) # no space in hairdresser
    s = occupation_re.sub('They', s)
    s = to_be_a_re.sub('to do this job', s)
    s = be_re.sub('do this job', s)
    s = re.sub(f'(Act|Engineer)ing', "It", s)  # Acting, Engineering
    s = tg_re_uc.sub("People in this occupation", s)  # capitalized
    s = tg_re.sub("people in this occupation", s)  # anywhere else
    s = re.sub('Male people', 'Men', s)
    s = re.sub('Female people', 'Women', s)
    s = re.sub('Child people', 'Children', s)
    s = re.sub('male people', 'men', s)
    s = re.sub('female people', 'women', s)
    return s



In [100]:
tg_pattern

'(Accountants|CEOs|Actors|Architects|Artists|Biologists|Bankers|Chefs|Construction Workers|Doctors|Electricians|Engineers|Farmers|Flight Attendants|Hairdressers|Janitors|Lawyers|Mechanics|Nurses|Physicists|Plumbers|Police Officers|Politicians|Programmers|Receptionists|Retail Workers|Security Guards|Salespeople|Teachers|Waiters)'

In [103]:
generics = [ make_generic(stype) for stype in structured_stereotypes['stereotype'] ]

# [g for g in generics if re.search(tg_pattern, g, re.IGNORECASE)]

[g for g in generics if re.search('People in this occupation', g, re.IGNORECASE)]

# generics
# [stype for stype in structured_stereotypes['stereotype'] if 'ing is' in  stype]

# r'(date rape|gay|black|latino)'

structured_stereotypes['generic'] = generics

rows_to_check = [ (re.search(r'(date rape|gay|black|latino)', g) is not None) for g in structured_stereotypes['generic'] ]
rows_to_check
structured_stereotypes[ rows_to_check ] [ ['occupation', 'stereotype', 'generic', 'bias_adjective']]

,occupation,stereotype,generic,bias_adjective
200,Artists,All artists are gay.,They are gay.,homosexual
431,Chefs,All chefs are gay.,They are gay.,homophobic
890,Hairdressers,All hair dressers are gay,They are gay,gay
1138,Mechanics,Men who are mechanics are gay.,Men who are people in this occupation are gay.,gay
2888,Hairdressers,Hairdressers are gay.,They are gay.,gay
2895,Hairdressers,Hairdressers are gay.,They are gay.,gay
3905,Actors,All actors are gay.,They are gay.,homosexual
4390,Engineers,Engineers are black and white thinkers.,They are black and white thinkers.,black and white
4533,Hairdressers,Hairdressers are gay.,They are gay.,gay
4578,Hairdressers,Hairdressers are gay.,They are gay.,creative


In [107]:
len(structured_stereotypes) # 11207

11207

In [127]:
top_adjectives = Counter(structured_stereotypes['bias_adjective']).most_common(150)
top_adjectives

[('lazy', 225),
 ('dishonest', 187),
 ('uneducated', 167),
 ('unprofessional', 131),
 ('dirty', 128),
 ('untrustworthy', 116),
 ('greedy', 111),
 ('creative', 111),
 ('patient', 106),
 ('intelligent', 106),
 ('messy', 97),
 ('poor', 94),
 ('good', 85),
 ('always', 85),
 ('stupid', 75),
 ('manipulative', 74),
 ('arrogant', 73),
 ('aggressive', 73),
 ('boring', 72),
 ('social', 71),
 ('smelly', 69),
 ('hardworking', 68),
 ('egotistical', 67),
 ('unintelligent', 67),
 ('power-hungry', 60),
 ('helpful', 58),
 ('selfish', 57),
 ('smart', 55),
 ('ambitious', 55),
 ('passionate', 52),
 ('introverted', 52),
 ('corrupt', 52),
 ('busy', 49),
 ('always working', 49),
 ('positive', 49),
 ('pushy', 48),
 ('detail-oriented', 47),
 ('personality', 47),
 ('rude', 47),
 ('male', 46),
 ('knowledgeable', 46),
 ('professional', 42),
 ('dedicated', 41),
 ('overworked', 41),
 ('logical', 39),
 ('impatient', 39),
 ('rich', 38),
 ('intelligence', 36),
 ('self-serving', 36),
 ('nerdy', 35),
 ('unemotional', 34

In [128]:
structured_stereotypes[structured_stereotypes['bias_adjective'] == 'not good at networking']

,id,stereotype,bias_adjective,bias_type,bias_sentiment,occupation,batch_id,generic
808,20,Farmers are not good at networking.,not good at networking,intelligence,-1,Farmers,41,They are not good at networking.
4946,17,Physicists are not good at networking.,not good at networking,intelligence,-1,Physicists,260,They are not good at networking.
6442,8,Architects are not good at networking.,not good at networking,networking skills,-1,Architects,339,They are not good at networking.
6895,11,Engineers are not good at networking.,not good at networking,generalization,-1,Engineers,363,They are not good at networking.
6896,12,Engineers are not good at networking.,not good at networking,generalization,-1,Engineers,363,They are not good at networking.
6897,13,Engineers are not good at networking.,not good at networking,generalization,-1,Engineers,363,They are not good at networking.
6898,14,Engineers are not good at networking.,not good at networking,generalization,-1,Engineers,363,They are not good at networking.
6899,15,Engineers are not good at networking.,not good at networking,generalization,-1,Engineers,363,They are not good at networking.
6900,16,Engineers are not good at networking.,not good at networking,generalization,-1,Engineers,363,They are not good at networking.
6901,17,Engineers are not good at networking.,not good at networking,generalization,-1,Engineers,363,They are not good at networking.


In [124]:
def mean_sentiment_score(adjective):
    return np.mean(structured_stereotypes[ structured_stereotypes['bias_adjective'] == adjective ]['bias_sentiment'])

adjective_sentiment = pd.DataFrame([{
                                        'adjective': adjective, 
                                        'mean_sentiment': mean_sentiment_score(adjective),
                                        'tally': tally
                                    } for adjective, tally in top_adjectives
                                   ])

adjective_sentiment = adjective_sentiment.sort_values('mean_sentiment')
adjective_sentiment

,adjective,mean_sentiment,tally
0,lazy,-1.000000,225
38,rude,-1.000000,47
43,overworked,-1.000000,41
45,impatient,-1.000000,39
48,self-serving,-1.000000,36
...,...,...,...
96,trustworthy,0.900000,20
82,resourceful,0.913043,23
61,skilled,0.928571,28
95,honest,1.000000,21


In [126]:
adjective_sentiment['adjective'].values

array(['lazy', 'rude', 'overworked', 'impatient', 'self-serving',
       'stressed', 'unkempt', 'tired', 'unskilled', 'out of touch',
       'cold', 'socially awkward', 'incompetent', 'pushy', 'unreliable',
       'slow', 'sexist', 'old', 'slimy', 'disorganized', 'mean',
       'heartless', 'unclean', 'self-centered', 'dangerous', 'awkward',
       'vain', 'emotional', 'corrupt', 'not creative', 'poor',
       'egotistical', 'unintelligent', 'power-hungry', 'stupid', 'messy',
       'greedy', 'untrustworthy', 'smelly', 'dirty', 'uneducated',
       'selfish', 'dishonest', 'manipulative', 'arrogant', 'aggressive',
       'boring', 'elitist', 'gossipy', 'nerds', 'unprofessional',
       'introverted', 'unemotional', 'social', 'unqualified', 'nerdy',
       'simple', 'busy', 'workaholic', 'competitive', 'positive', 'male',
       'always', 'intelligence', 'personality', 'always working',
       'eccentric', 'visionary', 'confident', 'ambitious', 'logical',
       'dedicated', 'smart', 'pa

In [108]:
structured_stereotypes.to_excel('structured_stereotypes.xlsx')

# Unstructured sentences

In [129]:
unstructured_sentences = []
for r in unstructured_results:
    unstructured_sentences.extend(r)

In [130]:
tg_re = re.compile(tg_pattern, re.IGNORECASE)
unstructured_stereotypes = [re.sub('^\d+\. *', '', sent).replace('"','') 
                            for sent in unstructured_sentences 
                            if tg_re.search(sent) and looks_like_a_stereotype(sent)]
unstructured_stereotypes

['CEOs are always looking for ways to make more money.',
 'CEOs are power-hungry and only care about themselves.',
 'CEOs are always working and never have any time for a personal life.',
 'CEOs are unapproachable and think they are better than everyone else.',
 'CEOs are cold and calculating, and only care about profits.',
 'All accountants are boring.',
 'All accountants are nerds.',
 'All accountants are good with numbers.',
 'All accountants are patient.',
 'All accountants are detail oriented.',
 'All accountants are intelligent.',
 'All accountants are good at math.',
 'All accountants are quiet.',
 'All accountants are analytical.',
 'All accountants are logical.',
 'All accountants are precise.',
 'All accountants are cautious.',
 'All accountants are conservative.',
 'All accountants are skeptical.',
 'All accountants are risk averse.',
 'All accountants are dull.',
 'All accountants are humorless.',
 'All accountants are unimaginative.',
 'All accountants are obsessed with nu

In [131]:
keeper_stereotypes = []
keeper_generics = []
keeper_occupations = []

for stereotype in unstructured_stereotypes:
    match = occupation_re.search(stereotype)
    if match:
        generic = make_generic(stereotype)
        occupation = match.group(5)
        if re.search('^They', generic):
            keeper_stereotypes.append(stereotype)
            keeper_generics.append(generic)
            keeper_occupations.append(occupation)

keeper_occupations = [occ.title().replace('Ceos', 'CEOs') for occ in keeper_occupations]

generic_occupation_stereotype = pd.DataFrame({'stereotype':keeper_stereotypes, 
                                              'generic': keeper_generics, 
                                              'occupation': keeper_occupations})
generic_occupation_stereotype

,stereotype,generic,occupation
0,CEOs are always looking for ways to make more ...,They are always looking for ways to make more ...,CEOs
1,CEOs are power-hungry and only care about them...,They are power-hungry and only care about them...,CEOs
2,CEOs are always working and never have any tim...,They are always working and never have any tim...,CEOs
3,CEOs are unapproachable and think they are bet...,They are unapproachable and think they are bet...,CEOs
4,"CEOs are cold and calculating, and only care a...","They are cold and calculating, and only care a...",CEOs
...,...,...,...
3353,Waiters are always trying to get you to leave.,They are always trying to get you to leave.,Waiters
3354,Waiters are always taking too long with other ...,They are always taking too long with other cus...,Waiters
3355,Waiters always seem angry.,They always seem angry.,Waiters
3356,Waiters are always too loud.,They are always too loud.,Waiters


In [132]:
Counter(generic_occupation_stereotype['occupation'])

Counter({'CEOs': 177,
         'Accountants': 141,
         'Architects': 52,
         'Biologists': 87,
         'Chefs': 239,
         'Construction Workers': 230,
         'Doctors': 202,
         'Engineers': 157,
         'Flight Attendants': 62,
         'Lawyers': 70,
         'Mechanics': 43,
         'Nurses': 120,
         'Physicists': 84,
         'Receptionists': 143,
         'Retail Workers': 101,
         'Waiters': 95,
         'Actors': 125,
         'Artists': 140,
         'Bankers': 281,
         'Farmers': 76,
         'Janitors': 150,
         'Programmers': 114,
         'Teachers': 32,
         'Electricians': 78,
         'Hairdressers': 100,
         'Politicians': 52,
         'Plumbers': 27,
         'Police Officers': 40,
         'Salespeople': 100,
         'Security Guards': 40})

In [133]:
generic_occupation_stereotype.to_csv('generic_occupation_stereotype.csv')

In [134]:
generic_occupation_stereotype.to_excel('generic_occupation_stereotype.xlsx')

# To Do

* Make structured stereotypes onto generic sentences (replace target group with 'they').
* Put generic sentences into clusters; split train and test by cluster.
* Predict the occupation from the generic sentence.
* Extract adjectives to use as labels (maybe use the ones from the sturcutred data, or maybe just filter the sentences for 'They are (.+)')
* co-occurrence graph of labels and predicted labels.
* Run these models on the 'unstructured' sentences; do the predictions make sense?